# TM10007 Assignment template

In [76]:
# Run this to use from colab environment
!git clone https://github.com/Doesjka/TM10007_ML_g9.git

fatal: destination path 'TM10007_ML_g9' already exists and is not an empty directory.


## Import packages

In [77]:
import os
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn import model_selection
from sklearn import decomposition
from sklearn import metrics
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVC
from sklearn.utils.fixes import loguniform
from sklearn.metrics import make_scorer, accuracy_score, recall_score
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest, f_classif


import seaborn
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import randint


## Define functions

In [78]:
def plot_learning_curve(estimator, title, X, y, axes, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate 3 plots: the test and training learning curve, the training
    samples vs fit times curve, the fit times vs score curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    axes : array of 3 axes, optional (default=None)
        Axes to use for plotting the curves.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 5-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """

    axes.set_title(title)
    if ylim is not None:
        axes.set_ylim(*ylim)
    axes.set_xlabel("Training examples")
    axes.set_ylabel("Score")

    train_sizes, train_scores, test_scores  = \
        model_selection.learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,
                       train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    # Plot learning curve
    axes.grid()
    axes.fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    axes.fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")
    axes.plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    axes.plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    axes.legend(loc="best")

    return plt

def custom_score(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred, average='macro')
    score = 2 * accuracy * recall / (accuracy + recall)
    return score

## Data loading

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [79]:
data = pd.read_csv('/content/TM10007_ML_g9/worclipo/Lipo_radiomicFeatures.csv', index_col=0)
print(f'The number of features: {len(data.columns)}')
print(f'The number of samples: {len(data.index)}')
data_punten = len(data.index) * len(data.columns)
print(data['label'])
ls = (data['label'] == 'liposarcoma').sum()

print(f'Of these samples {ls} are liposarcomas. That is {round(ls/len(data.index)*100)} percent.')

The number of features: 494
The number of samples: 115
ID
Lipo-001_0    liposarcoma
Lipo-002_0    liposarcoma
Lipo-003_0         lipoma
Lipo-004_0    liposarcoma
Lipo-005_0         lipoma
Lipo-006_0         lipoma
Lipo-007_0         lipoma
Lipo-008_0         lipoma
Lipo-009_0    liposarcoma
Lipo-010_0         lipoma
Lipo-011_0         lipoma
Lipo-012_0    liposarcoma
Lipo-013_0    liposarcoma
Lipo-014_0         lipoma
Lipo-015_0    liposarcoma
Lipo-016_0    liposarcoma
Lipo-017_0         lipoma
Lipo-018_0         lipoma
Lipo-019_0         lipoma
Lipo-020_0    liposarcoma
Lipo-021_0         lipoma
Lipo-022_0    liposarcoma
Lipo-023_0         lipoma
Lipo-024_0    liposarcoma
Lipo-025_0         lipoma
Lipo-026_0    liposarcoma
Lipo-027_0    liposarcoma
Lipo-028_0    liposarcoma
Lipo-029_0         lipoma
Lipo-030_0    liposarcoma
Lipo-031_0    liposarcoma
Lipo-032_0         lipoma
Lipo-033_0         lipoma
Lipo-034_0         lipoma
Lipo-035_0         lipoma
Lipo-036_0         lipoma
Lipo-0

## Splitting data in train and test set


In [80]:
y = data['label']

lb = preprocessing.LabelBinarizer()
y = lb.fit_transform(y)
y = y.flatten()

X = data.drop('label', axis=1)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

zero_variance = []

## Normaal verdeling en Variantie

### Normaal verdeling
Hieronder berekenen we hoe veel van de features normaal verdeeld zijn

In [81]:
aantal_normaal = 0

for column in X_train.columns:
    result = stats.shapiro(X_train[column])
    # print(result.pvalue)
    normaal = result.pvalue > 0.05
    aantal_normaal += normaal
    
print(aantal_normaal, " features zijn normaal verdeeld.")

88  features zijn normaal verdeeld.


/usr/local/lib/python3.9/dist-packages/scipy/stats/_morestats.py:1813: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "


### Variantie

In [82]:
variantie = X_train.var(axis=0)
variantie = variantie.sort_values()

# Haal features met variantie van 0 eruit, want die zeggen dus helemaal niks
zero_variance = variantie.keys()[variantie==0]
X_train = X_train.drop(zero_variance, axis=1)
print(f"{len(zero_variance)} features have a variance of zero. These features are deleted.")

21 features have a variance of zero. These features are deleted.


## Handling missing data 

### Throwing out features
All features that exist of at least 50% zeros are deleted from the data. 

In [83]:
zeros = (X_train == 0).sum()
threshold = 0.5 * len(y_train)
print('Threshold = ', threshold)
feature_del = zeros[zeros > threshold]

X_train = X_train.drop(columns=feature_del.index)
print(f'For {len(data.columns)-len(X_train.columns)-len(zero_variance)} features the data consisted of more than 50% zeros. These features are deleted.')

Threshold =  43.0
For 11 features the data consisted of more than 50% zeros. These features are deleted.


In [84]:
more_zeros = (X_train == 0).sum()
columns_zeros = more_zeros[more_zeros > 0].index
print(f'Of the remaining features, {len(columns_zeros)} features have at least one zero')
print(f'There is a total of {more_zeros.sum()} zeros left in the data')

Of the remaining features, 10 features have at least one zero
There is a total of 44 zeros left in the data


### Calculate number of missing values per sample

In [85]:
zeros_r = (X_train == 0).sum(axis=1)
threshold = 0.005 * X_train.size / len(y_train)
print('Threshold = ', threshold)
feature_del = zeros_r[zeros_r > threshold]
print(feature_del)

Threshold =  2.31
ID
Lipo-090_0    5
Lipo-095_0    3
Lipo-076_0    3
Lipo-003_0    3
dtype: int64


### Filling remaining zeros
All remaining zeros are replaced by the median of that feature. 

In [86]:
X_train_mean = X_train
X_train_median = X_train

print(X_train[columns_zeros])

for column in columns_zeros[:]:
    print('Kolom: ', column)
    column_mean = X_train.loc[X_train[column]!=0, column].mean()
    column_median = X_train.loc[X_train[column]!=0, column].median()
    # print('mean = ', column_mean)
    # print('median = ', column_median)

    # verschil2_percolumn = column_mean - column_median
    # print('Verschil tussen mean en median = ',verschil2_percolumn)

    # print('p-waarde normaalverdeling = ', stats.shapiro(X_train.loc[X_train[column]!=0, column]).pvalue)
    # print(' ')

    X_train_mean[column].replace(0, column_mean)
    X_train_median[column].replace(0, column_median)


verschil = abs(X_train_mean - X_train_median)
# print('Totaal verschil = ', verschil.sum().sum())

# We gaan voor de median
X_train = X_train_median

            PREDICT_original_sf_area_min_2.5D  \
ID                                              
Lipo-012_0                         183.736610   
Lipo-073_0                         185.128390   
Lipo-030_0                        1487.016678   
Lipo-042_0                          19.850316   
Lipo-100_0                         273.641968   
Lipo-008_0                         243.347168   
Lipo-034_0                        2148.103396   
Lipo-085_0                          53.412599   
Lipo-011_0                         109.910965   
Lipo-023_0                         115.013026   
Lipo-105_0                         129.150391   
Lipo-109_0                          54.355469   
Lipo-086_0                         558.160400   
Lipo-106_0                        1255.136481   
Lipo-080_0                         559.153608   
Lipo-107_0                          70.339765   
Lipo-066_0                        1060.228935   
Lipo-044_0                         300.091553   
Lipo-053_0          

## Outliers eruit halen


In [87]:
outliers_total = 0

for column in X_train.columns:
    q1 = X_train[column].quantile(0.25)
    q3 = X_train[column].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - (1.5 * iqr)
    upper_bound = q3 + (1.5 * iqr)

    outliers_column = (X_train[column] < lower_bound).sum() + (X_train[column] > upper_bound).sum()
    outliers_total += outliers_column

    X_train.loc[X_train[column] < lower_bound, column] = lower_bound
    X_train.loc[X_train[column] > upper_bound, column] = upper_bound

print(f"Er zijn {outliers_total} outliers vervangen")
print(data_punten)
print(f"Dit was {outliers_total / data_punten *100}% van het totale aantal datapunten")

Er zijn 2037 outliers vervangen
56810
Dit was 3.585636331631755% van het totale aantal datapunten


## Scaling

In [88]:
scaler = preprocessing.StandardScaler().fit(X_train_median)
X_train_scaled = scaler.transform(X_train_median)
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train_median.columns)

## ANOVA

In [89]:
# ANOVA feature selection for numeric input and categorical output
fs = SelectKBest(score_func=f_classif, k='all')

fit = fs.fit(X_train_scaled_df, y_train)
fit = fit.pvalues_
df = pd.DataFrame(fit, columns=['P-value'], index=X_train.columns)
rslt_df = df[df['P-value'] <= 0.05]
transpose = X_train_scaled_df.transpose()
new_df= rslt_df.join(transpose, how='left')
new_df = new_df.transpose()
X_train_ANOVA = new_df.drop('P-value', axis=0)
print(X_train_ANOVA)


    PREDICT_original_sf_compactness_std_2.5D  \
0                                   1.325789   
1                                  -0.987329   
2                                   1.044299   
3                                   1.160053   
4                                  -0.526039   
5                                  -0.814986   
6                                   0.715070   
7                                  -0.906895   
8                                  -1.543213   
9                                  -0.352070   
10                                  0.243738   
11                                  0.101556   
12                                  1.547984   
13                                 -0.998933   
14                                 -0.971318   
15                                  0.674629   
16                                  0.680154   
17                                  0.439095   
18                                  1.088633   
19                                  0.32

/usr/local/lib/python3.9/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [ 81  82  87  91  96 100] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.9/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


## PCA

In [90]:
pca = decomposition.PCA()
pca.fit(X_train_ANOVA)
X_pca = pca.transform(X_train_ANOVA)

component = 0
total_ratio = 0
while total_ratio < 0.95:
    total_ratio += pca.explained_variance_ratio_[component]
    component+=1

component2=0
while pca.explained_variance_ratio_[component2] > 0.001:
    component2+=1

print(component)
print(component2)
X_pca = X_pca[:,0:component]

17
33


## Initiatie classifiers

In [91]:
clsfs = list()
scoring = make_scorer(custom_score)

## Linear Classification

In [92]:
# Extract the best hyperparameters and fit
LDA_classifier = LinearDiscriminantAnalysis()
LDA_classifier.fit(X_pca, y_train)
clsfs.append(LDA_classifier)

## Quadratic Discriminant Analysis

In [93]:
# set parameters
parameters = {'reg_param': np.arange(0, 1, 0.1)}

# Specify the cross validation method to use, we use 10-fold stratified cross-validation
cv_10fold = model_selection.StratifiedKFold(n_splits=10)

# Create QDA object
qda = model_selection.RandomizedSearchCV(QuadraticDiscriminantAnalysis(), parameters, n_iter=11,
                                   cv=cv_10fold, scoring=scoring)

# Do the search
qda.fit(X_pca, y_train)

# Show the complete results of the cross validation
qda_df = pd.DataFrame(qda.cv_results_)
display(qda_df)

# Extract the best hyperparameters and fit
QDA_classifier = qda.best_estimator_
QDA_classifier.fit(X_pca, y_train)
clsfs.append(QDA_classifier)

# Whole search
# y_pred  = qda.predict(X_pca)



/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 10 is smaller than n_iter=11. Running 10 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_reg_param,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001634,0.000511,0.004829,0.003251,0.0,{'reg_param': 0.0},0.776386,0.565111,0.776386,0.565111,0.670807,0.788732,0.625,0.875,0.625,0.625,0.689253,0.101553,10
1,0.001028,0.000112,0.001710,0.000043,0.1,{'reg_param': 0.1},0.776386,0.670807,0.776386,0.670807,0.670807,0.788732,0.750,0.875,0.625,0.625,0.722893,0.078252,7
2,0.001337,0.000386,0.002265,0.000594,0.2,{'reg_param': 0.2},0.776386,0.670807,0.670807,0.670807,0.670807,0.788732,0.750,0.875,0.625,0.625,0.712335,0.077441,8
3,0.001375,0.000488,0.002187,0.000484,0.3,{'reg_param': 0.30000000000000004},0.776386,0.776386,0.552764,0.670807,0.670807,0.788732,0.750,0.875,0.625,0.750,0.723588,0.088663,5
4,0.001253,0.000287,0.002121,0.000396,0.4,{'reg_param': 0.4},0.776386,0.776386,0.552764,0.670807,0.670807,0.788732,0.750,0.875,0.625,0.750,0.723588,0.088663,5
5,0.001007,0.000027,0.001864,0.000250,0.5,{'reg_param': 0.5},0.776386,0.776386,0.434505,0.776386,0.658228,0.788732,0.750,0.750,0.500,0.750,0.696062,0.120398,9
6,0.001007,0.000040,0.001783,0.000155,0.6,{'reg_param': 0.6000000000000001},0.776386,0.776386,0.539846,0.881890,0.658228,0.788732,0.875,0.750,0.750,0.625,0.742147,0.101739,1
7,0.001029,0.000065,0.001743,0.000061,0.7,{'reg_param': 0.7000000000000001},0.776386,0.670807,0.539846,0.881890,0.658228,0.788732,0.875,0.750,0.750,0.625,0.731589,0.103107,2
8,0.001384,0.000446,0.002363,0.000720,0.8,{'reg_param': 0.8},0.776386,0.670807,0.539846,0.881890,0.658228,0.788732,0.750,0.750,0.875,0.625,0.731589,0.103107,2
9,0.001019,0.000035,0.001734,0.000034,0.9,{'reg_param': 0.9},0.776386,0.670807,0.539846,0.881890,0.658228,0.788732,0.750,0.750,0.875,0.625,0.731589,0.103107,2


## k-NN

In [94]:
from sklearn import neighbors
# Specify the search range, this could be multiple parameters for more complex classifiers
parameters = {'n_neighbors': randint(1, 6),
              'weights': ['uniform', 'distance'],
              'p': randint(1, 5),
              'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']}

# Specify the cross validation method to use, we use 10-fold stratified cross-validation
cv_10fold = model_selection.StratifiedKFold(n_splits=10)

# Create the grid search method, use area under ROC curve as scoring metric
# Too learn more about metrics see: https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
clf = model_selection.RandomizedSearchCV(neighbors.KNeighborsClassifier(), parameters, cv=cv_10fold, n_iter=500, scoring=scoring)

# Do the entire search
clf.fit(X_pca, y_train)
print("type = ", type(clf))
print(clf.best_estimator_)

# Show the complete results of the cross validation
clf_df = pd.DataFrame(clf.cv_results_)

# Extract the best k 
clf_df = clf_df.sort_values(by=['rank_test_score'])
# optimal_k = clf_df['param_n_neighbors'].iloc[0]
print(clf.best_params_)
print(clf.best_score_)

# Extract the best hyperparameters and fit
knn_classifier = clf.best_estimator_
knn_classifier.fit(X_pca, y_train)
clsfs.append(knn_classifier)

type =  <class 'sklearn.model_selection._search.RandomizedSearchCV'>
KNeighborsClassifier(algorithm='kd_tree')
{'algorithm': 'kd_tree', 'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}
0.7684445896093642


## Gedoe

In [95]:
# The scorers can be either one of the predefined metric strings or a scorer
# callable, like the one returned by make_scorer
scoring = {"accuracy": "accuracy", "recall": "recall"}

# Setting refit='AUC', refits an estimator on the whole dataset with the
# parameter setting that has the best cross-validated AUC score.
# That estimator is made available at ``gs.best_estimator_`` along with
# parameters like ``gs.best_score_``, ``gs.best_params_`` and
# ``gs.best_index_``
# gs = model_selection.RandomizedSearchCV(
#     neighbors.KNeighborsClassifier(),parameters,
#     scoring=["accuracy", "recall"],
#     refit="accuracy",
#     n_jobs=2,
#     return_train_score=True,
# )

gs = model_selection.RandomizedSearchCV(neighbors.KNeighborsClassifier(), parameters, cv=cv_10fold, n_iter=200, scoring=scoring, refit='accuracy')
gs.fit(X, y)
results = gs.cv_results_

KeyboardInterrupt: ignored

In [ ]:
print(results.keys())

plt.figure(figsize=(13, 13))
plt.title("GridSearchCV evaluating using multiple scorers simultaneously", fontsize=16)

plt.xlabel("n_neighbors")
plt.ylabel("Score")

ax = plt.gca()
# ax.set_xlim(0, 402)
# ax.set_ylim(0.73, 1)

# Get the regular numpy array from the MaskedArray
X_axis = np.array(results["param_n_neighbors"].data, dtype=float)

for scorer, color in zip(sorted(scoring), ["g", "k"]):
    sample = "test"
    style = "-"
    # for sample, style in (("test", "-")):
    sample_score_mean = results["mean_%s_%s" % (sample, scorer)]
    sample_score_std = results["std_%s_%s" % (sample, scorer)]
    ax.fill_between(
        X_axis,
        sample_score_mean - sample_score_std,
        sample_score_mean + sample_score_std,
        alpha=0.1 if sample == "test" else 0,
        color=color,
    )
    ax.plot(
        X_axis,
        sample_score_mean,
        style,
        color=color,
        alpha=1 if sample == "test" else 0.7,
        label="%s (%s)" % (scorer, sample),
    )

    best_index = np.nonzero(results["rank_test_%s" % scorer] == 1)[0][0]
    best_score = results["mean_test_%s" % scorer][best_index]

    # Plot a dotted vertical line at the best score for that scorer marked by x
    ax.plot(
        [
            X_axis[best_index],
        ]
        * 2,
        [0, best_score],
        linestyle="-.",
        color=color,
        marker="x",
        markeredgewidth=3,
        ms=8,
    )

    # Annotate the best score for that scorer
    ax.annotate("%0.2f" % best_score, (X_axis[best_index], best_score + 0.005))

plt.legend(loc="best")
plt.grid(False)
plt.show()

## Random Forest

In [ ]:
parameters = {'n_estimators': np.arange(50, 400, 50),
              'max_depth': [5, 10, 15],
              'min_samples_leaf': [2, 4]}

cv_10fold = model_selection.StratifiedKFold(n_splits=10)

clf = model_selection.RandomizedSearchCV(RandomForestRegressor(), parameters, cv=cv_10fold, n_iter=40, scoring=scoring)
    
# Fit the classifier
clf.fit(X_pca, y_train)

# Show the complete results of the cross validation
clf_df = pd.DataFrame(clf.cv_results_)
clf_df = clf_df.sort_values(by=['rank_test_score'])

# Extract the best hyperparameters 
print(clf.best_params_)
print(clf.best_score_)

# Extract the best hyperparameters and fit
RF_classifier = clf.best_estimator_
RF_classifier.fit(X_pca, y_train)
clsfs.append(RF_classifier)


## SVM

In [ ]:
parameters = {'C': loguniform(0.1, 100),
              'kernel': ['rbf', 'linear', 'poly', 'sigmoid'],
              'degree': randint(1, 5),
              'gamma': loguniform(1e-4, 1),
              'class_weight':['balanced', None]}
 
cv_10fold = model_selection.StratifiedKFold(n_splits=10)

clf = model_selection.RandomizedSearchCV(SVC(), parameters, cv=cv_10fold, n_iter=500, scoring=scoring)

# Do the entire search
clf.fit(X_pca, y_train)

# Show the complete results of the cross validation
clf_df = pd.DataFrame(clf.cv_results_)
clf_df = clf_df.sort_values(by=['rank_test_score'])

# Extract the best hyperparameters 
print(clf.best_score_)
print(clf.best_params_)

# Extract the best hyperparameters and fit
svm_classifier = clf.best_estimator_
svm_classifier.fit(X_pca, y_train)
clsfs.append(svm_classifier)


In [ ]:
print(clf_df)

## Learning curves

In [ ]:
fig = plt.figure(figsize=(24,10))
num = 0
for clf in clsfs:
    num +=1
    ax = fig.add_subplot(2, 3, num)
    title = str(type(clf))
    plot_learning_curve(clf, title, X_pca, y_train, ax, ylim=(0.3, 1.01), cv=10)
